<a href="https://colab.research.google.com/github/sccn/sound2meg/blob/main/Spatial_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import torch
import os
from torch.autograd import Variable
import math
import torch.nn as nn
from torch.nn.parameter import Parameter
import torch.optim as optim
from scipy.io import loadmat
from dataset_loading import Sound2MEGDataset
from torch.utils.data import Dataset, DataLoader, random_split
import gc

In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
import sys
sys.tracebacklimit = 0

In [6]:
class SpatialAttention(nn.Module):
  def __init__(self,in_channels, out_channels, K, path):
    super(SpatialAttention, self).__init__()
    self.out = out_channels
    self.input = in_channels
    self.K = K
    self.z = Parameter(torch.randn(self.out, K*K, dtype = torch.cfloat)/(32*32))
    self.z.requires_grad = True
    self.positions = loadmat(path + 'electrode_positions.mat')
    self.positions = self.positions['positions']
    self.x = torch.tensor(self.positions[:, 0]).to(device)
    self.y = torch.tensor(self.positions[:, 1]).to(device)
    self.cos_v = []
    self.sin_v = []
    self.cos = []
    self.sin = []
    for i in range(in_channels):
      self.cos_v = []
      self.sin_v = []
      for k in range(K):
        for l in range(K):
          self.cos_v.append(torch.cos(2*math.pi*(k*self.x[i]+l*self.y[i])))
          self.sin_v.append(torch.sin(2*math.pi*(k*self.x[i]+l*self.y[i])))
      self.cos.append(torch.stack(self.cos_v))
      self.sin.append(torch.stack(self.sin_v))
    self.cos = torch.stack(self.cos).to(device)
    self.sin = torch.stack(self.sin).to(device)
  def forward(self, X):
    N = X.size()[0]
    SA = torch.zeros(N, 270, 360).to(device)
    z_r = self.z.real
    z_i = self.z.imag
    a = (torch.mm(z_r.float(), torch.transpose(self.cos, 0, 1).float()) + torch.mm(z_i.float(), torch.transpose(self.sin, 0, 1).float())).to(device)
    exp2 = torch.sum(torch.exp(a[:, 0:self.out]), 1).to(device)
    exp2 = torch.transpose(exp2.unsqueeze(0), 0, 1)
    exp2 = torch.mm(exp2, torch.ones(1, 360).to(device))
    for i in range(N):
      exp1 = torch.mm(torch.exp(a), X[i]).to(device)
      SA[i] = (exp1/exp2).to(device)
      #SA[i] = SpatialAttentionSoftmax(self.input, self.out, X[i], a)
    return SA

In [7]:
class SubjectLayer(nn.Module):
  def __init__(self):
    super(SubjectLayer, self).__init__()
    self.layers = []

    for i in range(124): #124 subjects
      layer = nn.Conv2d(270, 270, 1).to(device)
      self.layers.append(layer)
      
  def forward(self, x, s_idx):
    x = x.unsqueeze(1)
    for i in range(len(x)):
      x[i] = self.layers[s_idx[i]](x[i].clone())
    return x[:, 0, :, :]

In [13]:
class Net(nn.Module):
  def __init__(self, path, F):
    super(Net, self).__init__()
    self.SA = SpatialAttention(273, 270, 32, path)
    self.Subject = SubjectLayer().to(device)
    self.F = F
  def forward(self, x, s_idx):
    x = self.SA(x).unsqueeze(0)
    x = torch.permute(x, (1, 2, 3, 0)) # subject attention?
    x = nn.Conv2d(270, 270, (1, 1)).to(device)(x)
    x = self.Subject(x, s_idx)
    for k in range(1,6):
      p = pow(2,(2*k)%5)
      q = pow(2,(2*k+1)%5)
      if k == 1:
        x = nn.Conv2d(270, 320, (3, 1), dilation = 1, padding = (1, 0)).to(device)(x)
        x = nn.BatchNorm2d(320).to(device)(x)
        x = nn.GELU().to(device)(x)
        x = nn.Conv2d(320, 320, (3, 1), dilation = 1, padding = (1, 0)).to(device)(x)
        x = nn.BatchNorm2d(320).to(device)(x)
        x = nn.GELU().to(device)(x)
        x = nn.Conv2d(320, 640, (3, 1), dilation = 2, padding = (2, 0)).to(device)(x)
        x = torch.transpose(x, 3, 1)
        x = nn.GLU().to(device)(x)
        x = torch.transpose(x, 3, 1)
      else:
        x1 = nn.Conv2d(320, 320, (3, 1), dilation = p, padding = (p, 0)).to(device)(x)
        x1 = nn.BatchNorm2d(320).to(device)(x1)
        x1 = nn.GELU().to(device)(x1)
        x2 = x + x1
        x3 = nn.Conv2d(320, 320, (3, 1), dilation = q, padding = (q, 0)).to(device)(x2)
        x3 = nn.BatchNorm2d(320).to(device)(x3)
        x3 = nn.GELU().to(device)(x3)
        x4 = x2 + x2
        x_out = nn.Conv2d(320, 640, (3, 1), dilation = 2, padding = (2, 0)).to(device)(x4)
        x_out = torch.transpose(x_out, 3, 1)
        x_out = nn.GLU().to(device)(x_out)
        x_out = torch.transpose(x_out, 3, 1)
    x_out = nn.Conv2d(320, 640, (1, 1)).to(device)(x_out)
    x_out = nn.GELU().to(device)(x_out)
    x_out = nn.Conv2d(640, self.F, (1, 1)).to(device)(x_out)
    return x_out

In [9]:
def CLIP_loss(Z, Y):
  N = Y.size(dim = 0)
  #inner_product = torch.zeros(N, N)
  log_softmax = torch.zeros(N).to(device)
  Z_row = torch.reshape(Z, (N, -1)).to(device)
  Y_row = torch.reshape(Y, (N, -1)).to(device)
  inner_product = (torch.mm(Z_row, torch.transpose(Y_row, 1, 0))/(N*N)).to(device)
  for i in range(N):
    inn = inner_product[i, :].to(device)
    log_softmax[i] = torch.log(nn.functional.softmax(inn, -1))[i]
  return sum(-1*log_softmax)

In [14]:
Dataset = Sound2MEGDataset('/content/drive/MyDrive/sound2meg/')
training_data, validation_data, test_data = random_split(Dataset, [11497, 3285, 1642], generator=torch.Generator().manual_seed(42))
Training_Data_Batches = DataLoader(training_data, batch_size = 2, shuffle = True)
Validation_Data_Batches = DataLoader(validation_data, batch_size = 128, shuffle = True)
BrainModule = Net('/content/drive/MyDrive/sound2meg/', 120)
BrainModule.to(device)
optimizer = optim.Adam(BrainModule.parameters(), lr = 0.0003)
loss_train = []
loss_val = []

for i in range(1):
  loss_t = 0
  loss_v = 0
  for MEG, WAV, Sub in Training_Data_Batches:
    Sub = Sub.tolist()
    Z = BrainModule(MEG.to(device), Sub)
    Z = Z[:, :, :, 0]
    loss = CLIP_loss(Z.float(), WAV.abs().float().to(device))
    print(loss.item())
    torch.autograd.set_detect_anomaly(True)
    optimizer.zero_grad()
    loss.backward()
    loss_t = loss_t + loss.item()
    optimizer.step()
  print(loss_t/len(Training_Data_Batches))
  loss_train.append(loss_t/len(Training_Data_Batches))
  for MEG_val, WAV_val, Sub_val in Validation_Data_Batches:
    Z_val = BrainModule(MEG_val.to(device), Sub_val)
    loss = CLIP_loss(Z_val.float(), WAV_val.abs().float().to(device))
    print(loss.item())
    loss_v = loss_v + loss.item()
  print(loss_v/len(Validation_Data_Batches))
  loss_val.append(loss_v/len(Validation_Data_Batches))
  gc.collect()
  torch.cuda.empty_cache()

print(loss_train)
print(loss_val)

1.391545057296753
1.3929414749145508
1.3853764533996582
1.393089771270752
1.3970651626586914
1.3805203437805176
1.3903281688690186
1.3848941326141357
1.3991668224334717
1.3873927593231201


KeyboardInterrupt: ignored